# Imports and Helper Functions

In [ ]:
import multiprocessing
num_available_cpus = multiprocessing.cpu_count()

print("Number of available CPUs:", num_available_cpus)

import sys

import math
import time
import tqdm

import numpy as np
import scipy as sp
from scipy import stats
from scipy.spatial import ConvexHull

import itertools
import logging
import matplotlib.pyplot as plt

import pandas as pd
import h5py

from sklearn import metrics

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
from torch.distributions import MultivariateNormal
import torch.utils.data as utils

from argparse import ArgumentParser
import re

sys.path.append("../new_flows")
from flows import RealNVP, Planar, MAF
from models import NormalizingFlowModel

In [ ]:
from nflows.flows.base import Flow
from nflows.flows.autoregressive import MaskedAutoregressiveFlow
from nflows.distributions.normal import StandardNormal
from nflows.transforms.base import CompositeTransform
from nflows.transforms.autoregressive import MaskedAffineAutoregressiveTransform, MaskedPiecewiseQuadraticAutoregressiveTransform, MaskedPiecewiseRationalQuadraticAutoregressiveTransform
from nflows.transforms.permutations import ReversePermutation

In [ ]:
from helper_functions import *

# Load and process bkg samples

In [ ]:
num_batches = 1

Mjj_cut = 1200
pt_cut = 550
eta_cut = None

In [ ]:
torch.cuda.empty_cache()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device =", device)
torch.set_default_tensor_type('torch.cuda.FloatTensor') if torch.cuda.is_available() else print ('cpu')

torch.set_num_threads(num_available_cpus)

print("Number of threads:", torch.get_num_threads())
print("Number of interop threads:", torch.get_num_interop_threads())

In [ ]:
bkg_data, bkg_unnorm_data, bkg_masses = LAPS_train(sample_type = 'qcdbkg', num_batches = num_batches)

bkg_mean = np.mean(bkg_unnorm_data, axis=0)
bkg_std = np.std(bkg_unnorm_data, axis=0)

In [ ]:
print(bkg_masses.shape)

plt.hist(bkg_masses, bins=50)
plt.show()

In [ ]:
total_PureBkg = torch.tensor(bkg_data)
total_PureBkg_selection = total_PureBkg

In [ ]:
total_PureBkg_selection.shape

In [ ]:
bs = 10000 * num_batches
bkgAE_train_iterator = utils.DataLoader(total_PureBkg_selection, batch_size=bs, shuffle=True) 
bkgAE_test_iterator = utils.DataLoader(total_PureBkg_selection, batch_size=bs)

# Build the bkg-trained model

In [ ]:
num_features = 14
hidden_features = 56

num_layers = 4
num_blocks_per_layer = 4
#num_iter = 10000
num_iter = 1000
print_interval = 20

#Current flow_type options: 'MAF', 'NSQUAD' (neural spline quadratic), 'NSRATQUAD' (neural spline rational quadratic)
flow_type = 'NSQUAD'

save_models = False

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device =", device)

In [ ]:
print(type(bkg_data[0,0]))

In [ ]:
bkg_loss_dict = dict()
bkg_flow_list = []

In [ ]:
filename = 'Pure_NF_%s_k%s_hf%s_nbpl%s' % (flow_type, num_layers, hidden_features, num_blocks_per_layer)

print("FCNN Hidden Layer Width: ", hidden_features)

print('------------------------------------')

bkg_base_dist = StandardNormal(shape=[num_features])

bkg_transforms = []
for _ in range(num_layers):
    bkg_transforms.append(ReversePermutation(features=num_features))
    if flow_type == 'MAF': 
        bkg_transforms.append(MaskedAffineAutoregressiveTransform(features=num_features, 
                                                          hidden_features=hidden_features))
    elif flow_type == 'NSQUAD': 
        bkg_transforms.append(MaskedPiecewiseQuadraticAutoregressiveTransform(features=num_features, 
                                                          hidden_features=hidden_features, tail_bound = 3.0, tails='linear'))
    elif flow_type == 'NSRATQUAD': 
        bkg_transforms.append(MaskedPiecewiseRationalQuadraticAutoregressiveTransform(features=num_features, 
                                                          hidden_features=hidden_features, tail_bound = 3.0, tails='linear'))

bkg_transform = CompositeTransform(bkg_transforms)

bkg_flow = Flow(bkg_transform, bkg_base_dist)
#print(bkg_flow)

bkg_optimizer = optim.Adam(bkg_flow.parameters())

bkg_tick = time.time()

bkg_min_loss = 999999
bkg_best_flow = None

cur_losses = []
patience_count = 0

for i in range(num_iter):
    
    terminate = False

    for batch_idx, x in enumerate(bkgAE_train_iterator):

        #x, y = datasets.make_moons(1024, noise=.1)
        #x = bkg_tr_data
        #x = torch.tensor(x, dtype=torch.float32)

        bkg_optimizer.zero_grad()
        loss = -bkg_flow.log_prob(inputs=x)[0].mean()
        loss.backward()
        bkg_optimizer.step()

        if batch_idx == len(bkgAE_train_iterator) - 1 :

            xline = torch.linspace(-2, 2)
            yline = torch.linspace(-2, 2)
            xgrid, ygrid = torch.meshgrid(xline, yline)
            xyinput = torch.cat([xgrid.reshape(-1, 1), ygrid.reshape(-1, 1)], dim=1)

            with torch.no_grad():
                #bkg_zgrid = bkg_flow.log_prob(xyinput)[0].exp().reshape(100, 100)
                bkg_zgrid = -bkg_flow.log_prob(x)[0].cpu().numpy()

                #print(bkg_zgrid.shape)
                #bkg_zgrid = bkg_zgrid[bkg_zgrid < 10]
                #print(bkg_zgrid.shape)
                #print(bkg_zgrid[:5])

            if (i + 1) % print_interval == 0: 
                print('bkg Iteration {} Complete'.format(i + 1))

            bkg_print_loss = loss.detach().cpu().numpy()
            cur_losses.append(bkg_print_loss) 
            print('Loss: ', bkg_print_loss)

            if bkg_print_loss < bkg_min_loss: 
                patience_count = 0
                bkg_best_flow = bkg_flow
                if save_models: 
                    torch.save(bkg_flow, "pure_flows/bkg_%s.pt" % (filename))
                bkg_min_loss = bkg_print_loss
                if (i + 1) % print_interval == 0: 
                    print('SAVING MODEL')
            else: 
                patience_count += 1
                if (i + 1) % print_interval == 0: 
                    print('NOT SAVING MODEL (PATIENCE = %s)' % patience_count)
                if patience_count == 10: 
                    terminate = True
                    break

            bkg_tock = time.time()

            if (i + 1) % print_interval == 0: 
                print('Time: ', bkg_tock - bkg_tick)
                print('------------------------------------')
                !nvidia-smi

            #plt.contourf(xgrid.numpy(), ygrid.numpy(), bkg_zgrid.numpy())
            #plt.title('iteration {}'.format(i + 1))
            #plt.show()

            #plt.hist(bkg_zgrid, bins=30, histtype='step')
            #plt.title('iteration {}'.format(i + 1))
            #plt.show()
            
    if terminate: 
        break

bkg_loss_dict[hidden_features] = float(bkg_min_loss)
bkg_flow_list.append(bkg_best_flow)

print('------------------------------------')

In [ ]:
plt.plot(cur_losses)
plt.xlabel('Epoch')
plt.ylabel('Bkg-trained loss')
plt.show()

# Load and process sig samples

In [ ]:
num_batches = 35

Mjj_cut = 1200
pt_cut = 550
eta_cut = None

In [ ]:
sig_data, sig_unnorm_data, sig_masses = LAPS_train(sample_type = 'wprimesig', num_batches = num_batches, inp_meanstd = (bkg_mean, bkg_std))

In [ ]:
print(sig_masses.shape)

plt.hist(sig_masses, bins=50)
plt.show()

In [ ]:
total_PureSig = torch.tensor(sig_data)
total_PureSig_selection = total_PureSig

In [ ]:
total_PureSig_selection.shape

In [ ]:
bs = 100 * num_batches
sigAE_train_iterator = utils.DataLoader(total_PureSig_selection, batch_size=bs, shuffle=True) 
sigAE_test_iterator = utils.DataLoader(total_PureSig_selection, batch_size=bs)

# Build the sig-trained model

In [ ]:
num_features = 14
hidden_features = 56

num_layers = 4
num_blocks_per_layer = 4
#num_iter = 10000
num_iter = 1000
print_interval = 20

#Current flow_type options: 'MAF', 'NSQUAD' (neural spline quadratic), 'NSRATQUAD' (neural spline rational quadratic)
flow_type = 'NSQUAD'

save_models = False

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device =", device)

In [ ]:
print(type(sig_data[0,0]))

In [ ]:
sig_loss_dict = dict()
sig_flow_list = []

In [ ]:
filename = 'Pure_NF_%s_k%s_hf%s_nbpl%s' % (flow_type, num_layers, hidden_features, num_blocks_per_layer)

print("FCNN Hidden Layer Width: ", hidden_features)

print('------------------------------------')

sig_base_dist = StandardNormal(shape=[num_features])

sig_transforms = []
for _ in range(num_layers):
    sig_transforms.append(ReversePermutation(features=num_features))
    if flow_type == 'MAF': 
        sig_transforms.append(MaskedAffineAutoregressiveTransform(features=num_features, 
                                                          hidden_features=hidden_features))
    elif flow_type == 'NSQUAD': 
        sig_transforms.append(MaskedPiecewiseQuadraticAutoregressiveTransform(features=num_features, 
                                                          hidden_features=hidden_features, tail_bound = 3.0, tails='linear'))
    elif flow_type == 'NSRATQUAD': 
        sig_transforms.append(MaskedPiecewiseRationalQuadraticAutoregressiveTransform(features=num_features, 
                                                          hidden_features=hidden_features, tail_bound = 3.0, tails='linear'))

sig_transform = CompositeTransform(sig_transforms)

sig_flow = Flow(sig_transform, sig_base_dist)
#print(sig_flow)

sig_optimizer = optim.Adam(sig_flow.parameters())

sig_tick = time.time()

sig_min_loss = 999999
sig_best_flow = None

cur_losses = []
patience_count = 0

for i in range(num_iter):
    
    terminate = False

    for batch_idx, x in enumerate(sigAE_train_iterator):

        #x, y = datasets.make_moons(1024, noise=.1)
        #x = sig_tr_data
        #x = torch.tensor(x, dtype=torch.float32)

        sig_optimizer.zero_grad()
        loss = -sig_flow.log_prob(inputs=x)[0].mean()
        loss.backward()
        sig_optimizer.step()

        if batch_idx == len(sigAE_train_iterator) - 1 :

            xline = torch.linspace(-2, 2)
            yline = torch.linspace(-2, 2)
            xgrid, ygrid = torch.meshgrid(xline, yline)
            xyinput = torch.cat([xgrid.reshape(-1, 1), ygrid.reshape(-1, 1)], dim=1)

            with torch.no_grad():
                #sig_zgrid = sig_flow.log_prob(xyinput)[0].exp().reshape(100, 100)
                sig_zgrid = -sig_flow.log_prob(x)[0].cpu().numpy()

                #print(sig_zgrid.shape)
                #sig_zgrid = sig_zgrid[sig_zgrid < 10]
                #print(sig_zgrid.shape)
                #print(sig_zgrid[:5])
                
            if (i + 1) % print_interval == 0: 
                print('sig Iteration {} Complete'.format(i + 1))

            sig_print_loss = loss.detach().cpu().numpy()
            cur_losses.append(sig_print_loss) 
            print('Loss: ', sig_print_loss)

            if sig_print_loss < sig_min_loss: 
                patience_count = 0
                sig_best_flow = sig_flow
                if save_models: 
                    torch.save(sig_flow, "pure_flows/sig_%s.pt" % (filename))
                sig_min_loss = sig_print_loss
                if (i + 1) % print_interval == 0: 
                    print('SAVING MODEL')
            else: 
                patience_count += 1
                if (i + 1) % print_interval == 0: 
                    print('NOT SAVING MODEL (PATIENCE = %s)' % patience_count)
                if patience_count == 10: 
                    terminate = True
                    break

            sig_tock = time.time()

            if (i + 1) % print_interval == 0: 
                print('Time: ', sig_tock - sig_tick)
                print('------------------------------------')
                !nvidia-smi

            #plt.contourf(xgrid.numpy(), ygrid.numpy(), sig_zgrid.numpy())
            #plt.title('iteration {}'.format(i + 1))
            #plt.show()

            #plt.hist(sig_zgrid, bins=30, histtype='step')
            #plt.title('iteration {}'.format(i + 1))
            #plt.show()
            
    if terminate: 
        break

sig_loss_dict[hidden_features] = float(sig_min_loss)
sig_flow_list.append(sig_best_flow)

print('------------------------------------')

In [ ]:
plt.plot(cur_losses)
plt.xlabel('Epoch')
plt.ylabel('Sig-trained Loss')
plt.show()

# Bkg-trained post-training analysis

In [ ]:
plt.rcParams["figure.figsize"] = (5,5)

In [ ]:
bkg_test = torch.tensor(bkg_data)
sig_test = torch.tensor(sig_data)

In [ ]:
def get_tpr_fpr_bkgtr(sigloss,bkgloss):
    bins = np.linspace(0,100,10001)
    tpr = []
    fpr = []
    for cut in bins:
        tpr.append(np.where(sigloss>cut)[0].shape[0]/len(sigloss))
        fpr.append(np.where(bkgloss>cut)[0].shape[0]/len(bkgloss))

    return tpr,fpr

In [ ]:
new_bkg_flow_list = []

new_bkg_loss_dict = dict()

filename = 'Pure_NF_%s_k%s_hf%s_nbpl%s' % (flow_type, num_layers, hidden_features, num_blocks_per_layer)

new_bkg_flow = torch.load("pure_flows/bkg_%s.pt" % (filename))

new_bkg_flow_list.append(new_bkg_flow)

new_bkg_loss = -new_bkg_flow.log_prob(bkg_data)[0].mean().detach().cpu().numpy()
new_bkg_loss_dict[hidden_features] = new_bkg_loss

print(bkg_loss_dict)
print(new_bkg_loss_dict)

In [ ]:
num_samples_tr = int(bkg_data.shape[0])

with torch.no_grad(): 
    bkg_samples = new_bkg_flow_list[-1].sample(num_samples_tr).detach().cpu().numpy()
#bkg_samples = new_bkg_flow_list[0].sample(num_samples_tr).detach().cpu().numpy()
print(bkg_samples.shape)
bkg_samples_bad_indices = np.argwhere((bkg_samples < -10) | (bkg_samples > 10))[:,0]
print(bkg_samples_bad_indices[:25])
new_bkg_samples = np.delete(bkg_samples, bkg_samples_bad_indices, axis = 0)

print(new_bkg_samples.shape)

print("Hidden Features: ", hidden_features)

plot_titles = [r'$M_{j1}$', r'Jet 1 $\tau_{21}$', r'Jet 1 $\tau_{32}$', r'Jet 1 $\tau_{43}$', r'Jet 1 $\tau_s$', r'Jet 1 $P_b$', r'Jet 1 $n_{pf}$', 
              r'$M_{j2}$', r'Jet 2 $\tau_{21}$', r'Jet 2 $\tau_{32}$', r'Jet 2 $\tau_{43}$', r'Jet 2 $\tau_s$', r'Jet 2 $P_b$', r'Jet 2 $n_{pf}$',]

for index in range(num_features): 
    n, bins, patches = plt.hist(bkg_data[:, index], bins=50, histtype='step', label='Input distribution')
    plt.hist(new_bkg_samples[:, index], bins=bins, histtype='step', label='NF estimated density')
    if index % 7 == 4: 
        plt.legend(loc=(1.04,0.85))
    plt.title(plot_titles[index])
    plt.show()
    
#plt.hist outputs binning, pass that as input to make binning the same for two hists

In [ ]:
bkgtr_bkgloss = -new_bkg_flow.log_prob(bkg_data)[0].detach().cpu().numpy()
bkgtr_sigloss = -new_bkg_flow.log_prob(sig_data)[0].detach().cpu().numpy()
print(bkgtr_bkgloss.shape)
print(bkgtr_sigloss.shape)

In [ ]:
tpr, fpr = get_tpr_fpr_bkgtr(bkgtr_sigloss, bkgtr_bkgloss)
tpr_np, fpr_np = np.array(tpr), np.array(fpr)

nonzero_idx = np.nonzero(fpr_np)

tpr_inverse = tpr_np[nonzero_idx]
fpr_inverse = 1/fpr_np[nonzero_idx]

In [ ]:
plt.plot(tpr_inverse,fpr_inverse)
plt.xlabel(r'$\epsilon_{sig}$',fontsize=15)
plt.ylabel(r'$1/\epsilon_{bkg}$',fontsize=15)
plt.yscale('log')
plt.title('Bkg-trained Pure NF Model')
plt.show()

In [ ]:
bkgtr_auc = metrics.auc(fpr,tpr)
plt.plot(fpr,tpr)
plt.xlabel(r'$\epsilon_{bkg}$',fontsize=15)
plt.ylabel(r'$\epsilon_{sig}$',fontsize=15)
plt.title('Bkg-trained Pure NF Model (AUC = %s)' % round(bkgtr_auc,3))
plt.show()

# Sig-trained post-training analysis

In [ ]:
def get_tpr_fpr_sigtr(sigloss,bkgloss):
    bins = np.linspace(0,100,10001)
    tpr = []
    fpr = []
    for cut in bins:
        tpr.append(np.where(sigloss<cut)[0].shape[0]/len(sigloss))
        fpr.append(np.where(bkgloss<cut)[0].shape[0]/len(bkgloss))

    return tpr,fpr

In [ ]:
new_sig_flow_list = []

new_sig_loss_dict = dict()

filename = 'Pure_NF_%s_k%s_hf%s_nbpl%s' % (flow_type, num_layers, hidden_features, num_blocks_per_layer)

new_sig_flow = torch.load("pure_flows/sig_%s.pt" % (filename))

new_sig_flow_list.append(new_sig_flow)

new_sig_loss = -new_sig_flow.log_prob(sig_data)[0].mean().detach().cpu().numpy()
new_sig_loss_dict[hidden_features] = new_sig_loss

print(sig_loss_dict)
print(new_sig_loss_dict)

In [ ]:
num_samples_tr = int(sig_data.shape[0])

with torch.no_grad(): 
    sig_samples = new_sig_flow_list[-1].sample(num_samples_tr).detach().cpu().numpy()
#sig_samples = new_sig_flow_list[0].sample(num_samples_tr).detach().cpu().numpy()
print(sig_samples.shape)
sig_samples_bad_indices = np.argwhere((sig_samples < -10) | (sig_samples > 10))[:,0]
print(sig_samples_bad_indices[:25])
new_sig_samples = np.delete(sig_samples, sig_samples_bad_indices, axis = 0)

print(new_sig_samples.shape)

print("Hidden Features: ", hidden_features)

plot_titles = [r'$M_{j1}$', r'Jet 1 $\tau_{21}$', r'Jet 1 $\tau_{32}$', r'Jet 1 $\tau_{43}$', r'Jet 1 $\tau_s$', r'Jet 1 $P_b$', r'Jet 1 $n_{pf}$', 
              r'$M_{j2}$', r'Jet 2 $\tau_{21}$', r'Jet 2 $\tau_{32}$', r'Jet 2 $\tau_{43}$', r'Jet 2 $\tau_s$', r'Jet 2 $P_b$', r'Jet 2 $n_{pf}$',]

for index in range(num_features): 
    n, bins, patches = plt.hist(sig_data[:, index], bins=50, histtype='step', label='Input distribution')
    plt.hist(new_sig_samples[:, index], bins=bins, histtype='step', label='NF estimated density')
    if index % 7 == 4: 
        plt.legend(loc=(1.04,0.85))
    plt.title(plot_titles[index])
    plt.show()
    
#plt.hist outputs binning, pass that as input to make binning the same for two hists

In [ ]:
sigtr_bkgloss = -new_sig_flow.log_prob(bkg_data)[0].detach().cpu().numpy()
sigtr_sigloss = -new_sig_flow.log_prob(sig_data)[0].detach().cpu().numpy()
print(sigtr_bkgloss.shape)
print(sigtr_sigloss.shape)

In [ ]:
tpr, fpr = get_tpr_fpr_sigtr(sigtr_sigloss, sigtr_bkgloss)
tpr_np, fpr_np = np.array(tpr), np.array(fpr)

nonzero_idx = np.nonzero(fpr_np)

tpr_inverse = tpr_np[nonzero_idx]
fpr_inverse = 1/fpr_np[nonzero_idx]

In [ ]:
plt.plot(tpr_inverse,fpr_inverse)
plt.xlabel(r'$\epsilon_{sig}$',fontsize=15)
plt.ylabel(r'$1/\epsilon_{bkg}$',fontsize=15)
plt.yscale('log')
plt.title('Sig-trained Pure NF Model')
plt.show()

In [ ]:
sigtr_auc = metrics.auc(fpr,tpr)
plt.plot(fpr,tpr)
plt.xlabel(r'$\epsilon_{bkg}$',fontsize=15)
plt.ylabel(r'$\epsilon_{sig}$',fontsize=15)
plt.title('Sig-trained Pure NF Model (AUC = %s)' % round(sigtr_auc,3))
plt.show()

# 2D ROC Curves (Still in Development)

In [ ]:
def get_hull_coordinates(scan_tpr, scan_fpr):
    points = np.array([scan_fpr,scan_tpr])
    points = points.transpose()
    hull = ConvexHull(points)
    fpr = np.array(points[hull.vertices[:],0])
    tpr = np.array(points[hull.vertices[:],1])
    nonzero_idx = np.nonzero(fpr)
    fpr = fpr[nonzero_idx]
    tpr = tpr[nonzero_idx]
    return tpr, fpr 

def get_tpr_fpr_2d(sigae_sigloss,sigae_bkgloss,bkgae_sigloss,bkgae_bkgloss):
    bins_sigae = np.arange(0,30,0.1)
    bins_bkgae = np.arange(0,30,0.1)
    tpr = []
    fpr = []
    for sigcut in bins_sigae:
        for bkgcut in bins_bkgae:
            tpr_t = np.where((sigae_sigloss<sigcut)&(bkgae_sigloss>bkgcut))[0].shape[0]/len(sigae_sigloss)
            fpr_t = np.where((sigae_bkgloss<sigcut)&(bkgae_bkgloss>bkgcut))[0].shape[0]/len(sigae_bkgloss)
            tpr.append(tpr_t)
            fpr.append(fpr_t)
            
    tpr, fpr = get_hull_coordinates(tpr, fpr)
                

    return tpr,fpr

In [ ]:
tpr2dlist = []
fpr2dlist = []
tpr2dlist_forinverse = []
fpr2dinverselist = []

SSLs = sigtr_sigloss
SBLs = sigtr_bkgloss
BSLs = bkgtr_sigloss
BBLs = bkgtr_bkgloss

print(SSLs[:20])
print(SBLs[:20])
print(BSLs[:20])
print(BBLs[:20])

tpr2d, fpr2d = get_tpr_fpr_2d(SSLs, SBLs, BSLs, BBLs)
print(tpr2d.shape)
print(fpr2d.shape)

tpr2d_np, fpr2d_np = np.array(tpr2d), np.array(fpr2d)

nonzero_idx = np.nonzero(fpr2d_np)

tpr2d_inverse = tpr2d_np[nonzero_idx]
fpr2d_inverse = 1/fpr2d_np[nonzero_idx]

In [ ]:
plt.plot(tpr2d_inverse,fpr2d_inverse)
plt.xlabel(r'$\epsilon_{sig}$',fontsize=15)
plt.ylabel(r'$1/\epsilon_{bkg}$',fontsize=15)
plt.yscale('log')
plt.title('2D ROC Curve (Pure NF Models)')
plt.show()

In [ ]:
auc2d = metrics.auc(fpr2d,tpr2d)
plt.plot(fpr2d,tpr2d)
plt.xlabel(r'$\epsilon_{bkg}$',fontsize=15)
plt.ylabel(r'$\epsilon_{sig}$',fontsize=15)
plt.title('2D ROC Curve (Pure NF Models)' % auc2d)
plt.show()

# QUAK spaces

In [ ]:
plt.rcParams["figure.figsize"] = (10,10)

In [ ]:
temp_bkgtr_bkgloss = np.append(bkgtr_bkgloss, np.array([0,]))
temp_sigtr_bkgloss = np.append(sigtr_bkgloss, np.array([0,]))
temp_bkgtr_sigloss = np.append(bkgtr_sigloss, np.array([0,]))
temp_sigtr_sigloss = np.append(sigtr_sigloss, np.array([0,]))

In [ ]:
def multidim_cumsum(a):
    out = a.cumsum(-1)
    for i in range(2,a.ndim+1):
        np.cumsum(out, axis=-i, out=out)
    return out

In [ ]:
def plot_cdf(inp_h_bkg, inp_h_sig, inp_X, inp_Y, inp_plot_lower_bound = None, inp_plot_upper_bound = None, num_bins = 50, save_cdfs = False): 

    bkg_cumulative_counts = multidim_cumsum(inp_h_bkg)
    sig_cumulative_counts = multidim_cumsum(inp_h_sig)

    cumulative_counts = sig_cumulative_counts/np.sqrt(bkg_cumulative_counts)
    flattened_cumulative_counts = cumulative_counts.flatten()

    h_bkg_sig, _, _, _ = plt.hist2d(inp_X, inp_Y, weights=flattened_cumulative_counts, cmap = plt.cm.jet, bins=num_bins)
    plt.colorbar()
    plt.xlabel("bkg-Trained Model Loss")
    plt.ylabel("sig-Trained Model Loss")
    plt.title("S/sqrt(B) 2D Loss Histogram (CDF)")
    
    if inp_plot_lower_bound is not None and inp_plot_upper_bound is not None: 
        plt.xlim(inp_plot_lower_bound, inp_plot_upper_bound)
        plt.ylim(inp_plot_lower_bound, inp_plot_upper_bound)
    
    if save_cdfs: 
        print(filename)
        plt.savefig('cdfs/%s.png' % (filename))
    plt.show()

In [ ]:
reco_loss_only = True
x_bad_loss_cutoff = 50
y_bad_loss_cutoff = 50
    
#------------------------------------------------------------------------------

num_bins = 2000

h_bkg, bkg_xedges, bkg_yedges, _ = plt.hist2d(temp_bkgtr_bkgloss, temp_sigtr_bkgloss, cmap = plt.cm.jet, bins=num_bins)
plt.colorbar()
plt.xlabel("BKG-Trained Model Loss")
plt.ylabel("SIG-Trained Model Loss")
plt.title("BKG Training Data 2D Loss Histogram (PDF)")
plt.xlim(0, x_bad_loss_cutoff)
plt.ylim(0, y_bad_loss_cutoff)
plt.show()

#------------------------------------------------------------------------------

num_bins = 250

h_sig, sig_xedges, sig_yedges, _ = plt.hist2d(temp_bkgtr_sigloss, temp_sigtr_sigloss, cmap = plt.cm.jet, bins=num_bins)
plt.colorbar()
plt.xlabel("BKG-Trained Model Loss")
plt.ylabel("SIG-Trained Model Loss")
plt.title("SIG Training Data 2D Loss Histogram (PDF)")
plt.xlim(0, x_bad_loss_cutoff)
plt.ylim(0, y_bad_loss_cutoff)
plt.show()

#------------------------------------------------------------------------------

num_bins = 1000
save_cdfs = False

x_lowest_bin_center = np.min(bkg_xedges) + (bkg_xedges[1]-bkg_xedges[0]) / 2

x_highest_bin_center = np.max(bkg_xedges) - (bkg_xedges[1]-bkg_xedges[0]) / 2
x_bin_step = (bkg_xedges[1]-bkg_xedges[0])

y_lowest_bin_center = np.min(bkg_yedges) + (bkg_yedges[1]-bkg_yedges[0]) / 2
y_highest_bin_center = np.max(bkg_yedges) - (bkg_yedges[1]-bkg_yedges[0]) / 2
y_bin_step = (bkg_yedges[1]-bkg_yedges[0])

h_bkg, bkg_xedges, bkg_yedges, _ = plt.hist2d(temp_bkgtr_bkgloss, temp_sigtr_bkgloss, cmap = plt.cm.jet, bins=num_bins)
h_sig, sig_xedges, sig_yedges, _ = plt.hist2d(temp_bkgtr_sigloss, temp_sigtr_sigloss, cmap = plt.cm.jet, bins=num_bins)
#print(h_bkg)
#print(h_skg)

counts = h_sig/np.sqrt(h_bkg)
flattened_counts = counts.flatten()

print(num_bins*x_bin_step)

X,Y = np.mgrid[x_lowest_bin_center:x_highest_bin_center+x_bin_step:x_bin_step, y_lowest_bin_center:y_highest_bin_center+y_bin_step:y_bin_step]
X = X[:num_bins, :num_bins]
Y = Y[:num_bins, :num_bins]
X = X.flatten()
Y = Y.flatten()

print(X.shape)
print(Y.shape)
print(counts.shape)
print(flattened_counts.shape)

h_bkg_sig, _, _, _ = plt.hist2d(X, Y, weights=flattened_counts, cmap = plt.cm.jet, bins=num_bins)
plt.colorbar()
plt.xlabel("BKG-Trained Model Loss")
plt.ylabel("SIG-Trained Model Loss")
plt.title("S/sqrt(B) 2D Loss Histogram (PDF)")
plt.xlim(0, x_bad_loss_cutoff)
plt.ylim(0, y_bad_loss_cutoff)
plt.show()

#------------------------------------------------------------------------------

plot_cdf(h_bkg, h_sig, X, Y, 0, x_bad_loss_cutoff, num_bins=num_bins, save_cdfs=save_cdfs)

# Bump Hunt CSV File Converter

In [ ]:
num_bkg_batches = 2
num_sig_batches = 1

sig_sampling_percentage = 100

In [ ]:
plt.rcParams["figure.figsize"] = (5,5)

In [ ]:
bkg_data, bkg_unnorm_data, bkg_masses = LAPS_test(sample_type = 'qcdbkg', num_batches = num_bkg_batches)

bkg_mean = np.mean(bkg_unnorm_data, axis=0)
bkg_std = np.std(bkg_unnorm_data, axis=0)

In [ ]:
sig_data, sig_unnorm_data, sig_masses = LAPS_test(sample_type = 'wprimesig', num_batches = num_sig_batches, inp_meanstd = (bkg_mean, bkg_std))

sig_num_samples = sig_data.shape[0]
sampling_indices = np.random.randint(sig_num_samples, size = int(sig_sampling_percentage * sig_num_samples / 100))
sig_data = sig_data[sampling_indices, :]
sig_unnorm_data = sig_unnorm_data[sampling_indices, :]
sig_masses = sig_masses[sampling_indices, :]

In [ ]:
num_bkg_samples = bkg_data.shape[0]
num_sig_samples = sig_data.shape[0]
sig_sample_percentage = 100 * num_sig_samples / (num_bkg_samples + num_sig_samples)

bkg_data_WL = np.concatenate((bkg_data, np.zeros((num_bkg_samples,1), dtype='float32')), axis=1)
sig_data_WL = np.concatenate((sig_data, np.ones((num_sig_samples,1), dtype='float32')), axis=1)

test_data = np.concatenate((bkg_data_WL, sig_data_WL), axis=0)
test_masses = np.concatenate((bkg_masses, sig_masses), axis=0)
#test_data = bkg_data_WL
#test_masses = bkg_masses
test_data_WM = np.concatenate((test_masses, test_data), axis=1)

np.random.shuffle(test_data_WM)

test_masses = test_data_WM[:,0]
test_labels = test_data_WM[:,-1]
test_data = test_data_WM[:,1:-1]

num_test_samples = test_data.shape[0]

print("Num bkg samples: ", num_bkg_samples)
print("Num sig samples: ", num_sig_samples)
print("Num test samples: ", num_test_samples)
print("Signal Percentage: ", sig_sample_percentage)

In [ ]:
df_mass = pd.DataFrame(np.ndarray.tolist(test_masses))
df_mass.to_csv('csv_files/test_masses.csv')

In [ ]:
filename = 'Pure_NF_%s_k%s_hf%s_nbpl%s' % (flow_type, num_layers, hidden_features, num_blocks_per_layer)
bkg_model = torch.load("pure_flows/bkg_%s.pt" % (filename))
sig_model = torch.load("pure_flows/sig_%s.pt" % (filename))

In [ ]:
bkg_loss_indices = np.where(test_labels==0)
sig_loss_indices = np.where(test_labels==1)

In [ ]:
bkgtr_test_losses = -bkg_model.log_prob(test_data)[0].detach().cpu().numpy()
print(bkgtr_test_losses.shape)

In [ ]:
bkgtr_bkg_losses = bkgtr_test_losses[bkg_loss_indices]
print(bkgtr_bkg_losses.shape)

bkgtr_sig_losses = bkgtr_test_losses[sig_loss_indices]
print(bkgtr_sig_losses.shape)

In [ ]:
df_bkgloss = pd.DataFrame(np.ndarray.tolist(bkgtr_test_losses))
df_bkgloss.to_csv('csv_files/bkgtr_test_losses.csv')

In [ ]:
sigtr_test_losses = -sig_model.log_prob(test_data)[0].detach().cpu().numpy()
print(sigtr_test_losses.shape)

In [ ]:
sigtr_bkg_losses = sigtr_test_losses[bkg_loss_indices]
print(sigtr_bkg_losses.shape)

sigtr_sig_losses = sigtr_test_losses[sig_loss_indices]
print(sigtr_sig_losses.shape)

In [ ]:
df_sigloss = pd.DataFrame(np.ndarray.tolist(sigtr_test_losses))
df_sigloss.to_csv('csv_files/sigtr_test_losses.csv')

In [ ]:
bkgloss_cut = 15
low_bkgloss_indices = np.where(bkgtr_test_losses > bkgloss_cut)[0]
for index in range(2,8):  
    
    sigloss_cut = 5*index
    low_sigloss_indices = np.where(sigtr_test_losses < sigloss_cut)[0]
    low_loss_indices = np.intersect1d(low_bkgloss_indices, low_sigloss_indices)
    low_loss_test_masses = test_masses[low_loss_indices]
    low_loss_bkg_masses = test_masses[np.intersect1d(low_loss_indices, bkg_loss_indices)]
    
    n, bins, patches = plt.hist(low_loss_test_masses, bins=50, histtype = 'step', label = 'bkg + sig')
    plt.hist(low_loss_bkg_masses, bins = bins, histtype = 'step', label = 'bkg only')
    
    plt.xlabel('bkgloss > %s, sigloss < %s' % (bkgloss_cut, sigloss_cut))
    plt.legend()
    plt.show()

In [ ]:
plt.rcParams["figure.figsize"] = (10,10)

In [ ]:
plt.scatter(bkgtr_bkg_losses, sigtr_bkg_losses, s=2, label = 'QCD bkg samples')
plt.scatter(bkgtr_sig_losses, sigtr_sig_losses, s=2, label = r'W$\rightarrow$WZ sig samples')
plt.xlim(0,100)
plt.ylim(0,100)
plt.xlabel("bkg-Trained Model Loss")
plt.ylabel("sig-Trained Model Loss")
plt.title("Testing Data QUAK Space")
plt.legend()
plt.show()

# Load and processs graviton samples

In [ ]:
num_batches = 1

In [ ]:
plt.rcParams["figure.figsize"] = (5,5)

In [ ]:
graviton_data, graviton_unnorm_data, graviton_masses = LAPS_test(sample_type = 'graviton', num_batches = num_batches, inp_meanstd = (bkg_mean, bkg_std))
print(graviton_data.shape[0])

In [ ]:
bkgtr_graviton_losses = -bkg_model.log_prob(graviton_data)[0].detach().cpu().numpy()
print(bkgtr_graviton_losses.shape)

In [ ]:
sigtr_graviton_losses = -sig_model.log_prob(graviton_data)[0].detach().cpu().numpy()
print(sigtr_graviton_losses.shape)

# Load and process Wkk samples

In [ ]:
num_batches = 1

In [ ]:
Wkk_data, Wkk_unnorm_data, Wkk_masses = LAPS_test(sample_type = 'Wkk', num_batches = num_batches, inp_meanstd = (bkg_mean, bkg_std))
print(Wkk_data.shape[0])

In [ ]:
bkgtr_Wkk_losses = -bkg_model.log_prob(Wkk_data)[0].detach().cpu().numpy()
print(bkgtr_Wkk_losses.shape)

In [ ]:
sigtr_Wkk_losses = -sig_model.log_prob(Wkk_data)[0].detach().cpu().numpy()
print(sigtr_Wkk_losses.shape)

# Master QUAK Space 

In [ ]:
plt.rcParams["figure.figsize"] = (10,10)

In [ ]:
plt.scatter(bkgtr_bkg_losses, sigtr_bkg_losses, s=2, label = 'QCD bkg samples')
plt.scatter(bkgtr_sig_losses, sigtr_sig_losses, s=2, label = 'W\' sig samples')
plt.scatter(bkgtr_graviton_losses, sigtr_graviton_losses, s=2, label = 'Graviton sig samples')
plt.scatter(bkgtr_Wkk_losses, sigtr_Wkk_losses, s=2, label = 'Wkk sig samples')
plt.xlim(0,100)
plt.ylim(0,100)
plt.xlabel("bkg-Trained Model Loss")
plt.ylabel("sig-Trained Model Loss")
plt.title("Testing Data QUAK Space")
plt.legend()
plt.show()